From __Datascience from Scratch__

In [1]:
from ds_f import *

In [44]:
def normal_approximation_to_binomial(n,p):
    """finds mu and sigma corresponding to a Binomial(n,p)"""
    mu = p*n
    sigma = math.sqrt(p*(1-p)*n)
    return mu,sigma

# the normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

# its above the threshold if its not below the threshold
def normal_probability_above(lo,mu=0,sigma=1):
    return 1-normal_cdf(lo,mu,sigma)

# its between if < hi and not < low
def normal_probability_between(lo,hi,mu=0,sigma=1):
    return normal_cdf(hi,mu,sigma) - normal_cdf(lo,mu,sigma)

# outside if not between
def normal_probability_outside(lo,hi,mu=0,sigma=1):
    return 1 - normal_probability_between(lo,hi,mu,sigma)

def normal_upper_bound(probability, mu=0,sigma=1):
    """returns the z for which P(Z <=z) = probability"""
    return inverse_normal_cdf(probability,mu,sigma)

def normal_lower_bound(probability, mu=0,sigma=1):
    """returns the z for which P(Z <=z) = probability"""
    return inverse_normal_cdf(1 - probability,mu,sigma)

def normal_two_sided_bounds(probability,mu=0,sigma=1):
    tail_probability = (1-probability) / 2
    
    upper_bound = normal_lower_bound(tail_probability,mu,sigma)

    lower_bound = normal_upper_bound(tail_probability,mu,sigma)
    
    return lower_bound, upper_bound

def two_sided_p_value(x,mu=0,sigma=1):
    if x>= mu:
        # if x is greater than mean, tail is greater than x
        return 2*normal_probability_above(x,mu,sigma)
    else:
        # if x is less than mean, tail is less than x
        return 2*normal_probability_below(x,mu,sigma)
    
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below


# demo run

def run_experiment():
    """flip coin 1000 times, True=heads, False=tails"""
    
    return [random.random()<.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads=len([flip for flip in experiment if flip])
    return num_heads<469 or num_heads >531

# example demos
def run_demo_0():
    extreme_value_count = 0
    r = 100000
    for _ in range(r):
        num_heads = sum(1 if random.random()<.5 else 0 for _ in range(1000))
        if num_heads >= 530 or num_heads <= 470:
            extreme_value_count += 1
    print(extreme_value_count / r)

def run_demo_1():
    random.seed(0)
    experiments = [run_experiment() for _ in range(1000)]
    num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

    print(num_rejections)

In [4]:
mu_0,sigma_0 = normal_approximation_to_binomial(1000,.5)
normal_two_sided_bounds(.95,mu_0,sigma_0)

In [7]:
lo,hi = normal_two_sided_bounds(.95,mu_0,sigma_0)
mu_1, sigma_1 = normal_approximation_to_binomial(1000,.55)
type_2_probability = normal_probability_between(lo,hi,mu_1,sigma_1)
power = 1- type_2_probability
power

In [11]:
hi = normal_upper_bound(.95,mu_0,sigma_0)
type_2_probability = normal_probability_below(hi,mu_1,sigma_1)
power = 1-type_2_probability
power

In [14]:
two_sided_p_value(529.5,mu_0,sigma_0)

0.06207721579598857

In [19]:
two_sided_p_value(531.5,mu_0,sigma_0)

0.046345287837786575

In [21]:
upper_p_value(524.5,mu_0,sigma_0)

0.06062885772582083

In [24]:
upper_p_value(526.5,mu_0,sigma_0)

0.04686839508859242

In [34]:
p_hat = 525/1000
mu = p_hat
sig = math.sqrt(p_hat * (1-p_hat)/1000)
sig

0.015791611697353755

In [53]:
def estimated_parameters(N,n):
    p = n/N
    sigma = math.sqrt(p*(1-p)/N)
    return p,sigma

def a_b_test_statistic(N_A,n_A,N_B,n_B):
    p_A, sigma_A = estimated_parameters(N_A,n_A)
    p_B, sigma_B = estimated_parameters(N_B,n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [55]:
z = a_b_test_statistic(1000,200,1000,180)
z

-1.1403464899034472

In [56]:
z = a_b_test_statistic(1000,200,1000,150)
two_sided_p_value(z)

0.003189699706216853

-2.948839123097944